In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pickle


class PickleableLink:
    def __init__(self, link_data):
        self.picklable_data = link_data
        
linkList = []

driver=webdriver.Chrome()
for i in range(1,3):
    driver.get(f"https://xe.chotot.com/mua-ban-xe-may-ha-noi?page={i}")
    links_raw = driver.find_elements(By.CLASS_NAME, "AdItem_adItem__gDDQT")
    for link in links_raw:
        http = link.get_attribute('href')
        pickleable_link = PickleableLink(http)
        linkList.append(pickleable_link)


with open("test", "wb") as fp:   #Pickling
    pickle.dump(linkList, fp)

In [7]:
import pickle

clean_list = []

class PickleableLink:
    def __init__(self, link_data):
        self.picklable_data = link_data

# Unpickle the data
with open("test", "rb") as fp:
    link_list = pickle.load(fp)

# Extract and print the URLs
for link_object in link_list:
    url = link_object.picklable_data
    modified_url = url.rstrip(']')
    clean_list.append(modified_url)

with open("clean_url", "wb") as fp:   #Pickling
    pickle.dump(clean_list, fp)

In [8]:
import json

class PickleableLink:
    def __init__(self, link_data):
        self.picklable_data = link_data

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, PickleableLink):
            return {'picklable_data': obj.picklable_data}
        return super().default(obj)

# Unpickle the data
with open("clean_url", "rb") as fp:
    link_list = pickle.load(fp)

# Serialize to JSON with the custom encoder
json_data = json.dumps(link_list, cls=CustomEncoder)

# Save JSON to a file
with open("cleaned_data.json", "w") as json_file:
    json_file.write(json_data)


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

with open("cleaned_data.json", "rb") as fp:   # Unpickling
    linkItem = json.load(fp)
#datacleaning duplicate cleaning...
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains

print(linkItem)
df = pd.DataFrame(columns = 
    ['title', 'price', 'timeStamp', 'hang','dong','namdangki','soKM','tinhTrang','loaiXe','dungTich','xuatXu','chinhSachBH','TrongLuong'])
driver=webdriver.Chrome()

one_time = True
for link in linkItem:
    
    driver.get(link)
    # button = driver.find_element(By.XPATH, '//*[@id="__next"]/div/header/div[2]')
    # button.click()
    if one_time:
        driver.implicitly_wait(3)
        ac = ActionChains(driver)
        ac.move_by_offset(100, 100).click().perform()
        one_time = False
    
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='AdDecriptionVeh_adTitle__vEuKD' and @itemprop='name']").text
    except:
        title = None

    try:
        price = driver.find_element(By.XPATH, "//span[contains(@class, 'AdDecriptionVeh_price__u_N83')]/span[@itemprop='price']").text
        #price = [int(p.replace(' đ', '').replace('.', '')) for p in prices]
    except:
        price = None
    
    button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[4]/div[1]/div/div[4]/div/div[4]/button')
    button.click()
    
    try:
        hang = driver.find_element(By.XPATH, "//a[@itemprop='motorbikebrand' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        hang = None

    try:
        dong = driver.find_element(By.XPATH, "//a[@itemprop='motorbikemodel' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        dong = None

    try:
        namdangki = driver.find_element(By.XPATH, "//span[@itemprop='regdate' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        namdangki = None

    try:
        soKM = driver.find_element(By.XPATH, "//span[@itemprop='mileage_v2' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        soKM = None

    try:
        tinhTrang = driver.find_element(By.XPATH, "//span[@itemprop='condition_ad' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        tinhTrang = None

    try:
        loaiXe = driver.find_element(By.XPATH, "//span[@itemprop='motorbiketype' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        loaiXe = None

    try:
        dungTich = driver.find_element(By.XPATH, "//span[@itemprop='motorbikecapacity' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        dungTich = None

    try:
        xuatXu = driver.find_element(By.XPATH, "//span[@itemprop='motorbikeorigin' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        xuatXu = None

    try:
        chinhSachBH = driver.find_element(By.XPATH, "//span[@itemprop='veh_warranty_policy' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        chinhSachBH = None

    try:
        TrongLuong = driver.find_element(By.XPATH, "//span[@itemprop='veh_unladen_weight' and contains(@class, 'AdParam_adParamValue__IfaYa')]").text
    except:
        TrongLuong = None
    df = pd.concat([df, pd.DataFrame({'title': [title], 'price': [price], 'hang': [hang], 'dong': [dong], 
                                  'namdangki': [namdangki], 'soKM': [soKM], 'tinhTrang': [tinhTrang],
                                  'loaiXe': [loaiXe], 'dungTich': [dungTich], 'chinhSachBH': [chinhSachBH],
                                  'TrongLuong': [TrongLuong]})], ignore_index=True)
    # df = df.append(
    #     {'title' : title, 'price' : price, 'hang' : hang,'dong' : dong ,'baohanh': baohanh,'mausac': mausac,'dungluong' : dungluong,'xuatxu' : xuatxu, 'tinhtrang' : tinhtrang},ignore_index = True)
    df.to_csv('Data_DAP.csv', index=False, encoding='utf-8')  
    print(price)
    print('------')

['https://xe.chotot.com/mua-ban-xe-may-quan-cau-giay-ha-noi/110555514.htm#px=SR-stickyad-[PO-1][PL-top', 'https://xe.chotot.com/mua-ban-xe-may-quan-hoang-mai-ha-noi/106646432.htm#px=SR-stickyad-[PO-2][PL-top', 'https://xe.chotot.com/mua-ban-xe-may-quan-nam-tu-liem-ha-noi/109133536.htm#px=SR-stickyad-[PO-3][PL-top', 'https://xe.chotot.com/mua-ban-xe-may-quan-nam-tu-liem-ha-noi/115153668.htm#px=SR-stickyad-[PO-4][PL-top', 'https://xe.chotot.com/mua-ban-xe-may-quan-nam-tu-liem-ha-noi/109564560.htm#px=SR-stickyad-[PO-5][PL-top', 'https://xe.chotot.com/mua-ban-xe-may-quan-cau-giay-ha-noi/85171432.htm', 'https://xe.chotot.com/mua-ban-xe-may-quan-cau-giay-ha-noi/114454248.htm', 'https://xe.chotot.com/mua-ban-xe-may-quan-nam-tu-liem-ha-noi/112980229.htm', 'https://xe.chotot.com/mua-ban-xe-may-huyen-thuong-tin-ha-noi/116336846.htm', 'https://xe.chotot.com/mua-ban-xe-may-quan-thanh-xuan-ha-noi/106154111.htm', 'https://xe.chotot.com/mua-ban-xe-may-quan-nam-tu-liem-ha-noi/95354987.htm', 'https://x

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[4]/div[1]/div/div[4]/div/div[4]/button"}
  (Session info: chrome=124.0.6367.201); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF729431522+60802]
	(No symbol) [0x00007FF7293AAC22]
	(No symbol) [0x00007FF729267CE4]
	(No symbol) [0x00007FF7292B6D4D]
	(No symbol) [0x00007FF7292B6E1C]
	(No symbol) [0x00007FF7292FCE37]
	(No symbol) [0x00007FF7292DABBF]
	(No symbol) [0x00007FF7292FA224]
	(No symbol) [0x00007FF7292DA923]
	(No symbol) [0x00007FF7292A8FEC]
	(No symbol) [0x00007FF7292A9C21]
	GetHandleVerifier [0x00007FF7297341BD+3217949]
	GetHandleVerifier [0x00007FF729776157+3488183]
	GetHandleVerifier [0x00007FF72976F0DF+3459391]
	GetHandleVerifier [0x00007FF7294EB8E6+823622]
	(No symbol) [0x00007FF7293B5FBF]
	(No symbol) [0x00